# Introduction
General housekeeping for running the notebooks and analysis pipeline.

In [44]:
import pickle as pkl
import scanpy as sc
import matplotlib as mpl
import os
import pandas as pd

In [2]:
mountpoint = '/data/clue/'

Path to the most recent concat object:

In [3]:
path_concat = mountpoint + 'prod/comb/h5ads/concat_6.h5ad'

Path to eQTL results:

In [26]:
path_eqtl = mountpoint + 'prod/eqtl/mateqtl/vals/'

# Create Metadata Dictionary

I use a `pkl` called `meta.pkl` in various notebooks throughout the processing. Here is where I create it.

In [15]:
meta = dict()

## `cts`

In [5]:
concat = sc.read_h5ad(path_concat)

In [16]:
ct_types = [i for i in concat.obs.columns if i.startswith('ct')]

In [17]:
meta['cts'] = dict()
for ct_type in ct_types:
    meta['cts'][ct_type] = concat.obs[ct_type].cat.categories.tolist()

## `ct` colors

In [18]:
ct_colors = dict()
for ct_type in ct_types:
    ct_colors[ct_type] = dict()
    ct_colors[ct_type]['hex'] = dict(zip(concat.obs[ct_type].cat.categories, concat.uns[ct_type + '_colors']))
    ct_colors[ct_type]['rgb'] = dict(zip(concat.obs[ct_type].cat.categories, list(map(mpl.colors.hex2color, concat.uns[ct_type + '_colors']))))

In [19]:
meta['ct_colors'] = ct_colors

## `conds`

In [20]:
meta['conds'] = dict()
meta['conds']['conds_all'] = ['0', 'A', 'B', 'C', 'G', 'P' ,'R']
meta['conds']['conds_filt'] = ['A', 'B', 'C', 'G', 'P' ,'R']
meta['conds']['conds_focus'] = ['A', 'B', 'C', 'G' ,'R']
meta['conds']['conds_stims'] = ['A', 'B', 'G' ,'R', 'P']

## `cond_colors`

In [21]:
cond_colors = dict()
conds_all = meta['conds']['conds_all'] # 
cond_colors['hex'] = dict(zip(conds_all, sc.pl.palettes.default_20))
cond_colors['rgb'] = dict(zip(conds_all, list(map(mpl.colors.hex2color, sc.pl.palettes.default_20))))

In [22]:
meta['cond_colors'] = cond_colors

## `eQTL`

In [35]:
output_files = [fn for fn in os.listdir(path_eqtl) if fn.startswith('mateqtl_')]

In [36]:
cond_cts = list()
for output_file in output_files:
    cond_cts.append(tuple(output_file.replace('mateqtl_', '').replace('_all_cis.csv', '').split('_', maxsplit=1)))

In [49]:
meta['eqtl'] = dict()
meta['eqtl']['cond_cts'] = pd.DataFrame(cond_cts, columns=['cond', 'ct'], dtype='category')

# Export

In [50]:
# with open(mountpoint + 'meta.pkl', 'wb') as file:
#     pkl.dump(meta, file)

In [36]:
with open(mountpoint + 'meta.pkl', 'rb') as file:
    meta = pkl.load(file)